In [1]:
from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
import random
from datetime import datetime

# General Strategy
This program will create a portfolio that will best meet the S&P 500 and TSX 60 Indexes. To achieve this, we decided to make a safe portfolio that matches the risk of both indexes. With an initial starting value of $1,000,000 CAD, we aim for the overall portfolio to move with the index. We have combined both indexes to compare them equally.

To create this portfolio, we had 3 main parameters used to rank each stock. The first parameter was tracking error which measures how much a stock deviates from an index. Ideally, we wanted to choose stocks with tracking errors as close to 0 as possible whether it be below or above.

Next, we analyzed stocks using Beta. Beta measures a stock’s volatility compared to an index’s volatility. We want to choose stocks with a Beta value as close to 1 as possible since a beta value of 1 represents the same volatility for both the index and the stock.

Lastly, we checked the correlation between each stock and the index. Correlation measures how 2 securities move in relation to one another. We want to choose stocks that have correlation values as close to 1 as possible. We want to avoid stocks with a negative correlation since they historically have had the reverse effect on trying to meet the market.

We will weigh these 3 parameters equally and rank our stocks with a score of 0-1. From then we will optimize our portfolio which will contain between 12-24 stocks based on the tracking error of our portfolio compared to the index.

# PART 1: Filtering out unusable stocks

## Filtering Ticker List

The first step is to go through all the ticker symbols from the "Tickers.csv" file and extract the stocks we have to work with. From here we have to clean the data set to remove any tickers that are not valid. 

Our filter function removes:

- Tickers that may have been delisted since they can't be bought in our portfolio
- Duplicate tickers since we should only consider unique stocks
- Tickers with less than an average monthly volume of 100,000 shares (We also removed any calendar months with less than 18 trading days as it would not be considered a complete representation of typical market activity, potentially leading to inaccurate or skewed calculations of average monthly trading volumes)
- Any tickers that do not have a valid stock denominated in either USD or CAD

As we filter through each ticker, non-valid stocks will be removed and a note will be added as to why it was removed. At the end, it will output a list of valid tickers we can further analyze under the name (valid_tickers)




In [6]:
# Extracts the Ticker symbols from a CSV file and creates a list that will be filtered out
file = 'Tickers.csv'
df = pd.read_csv(file, header=None)
tickers = df[0].tolist()
valid_tickers = [] # initialize the ticker list

# Define filtering parameters 

# Dates and data used to check historical average monthly volume
vol_start_date = '2023-10-01' 
vol_end_date = '2024-09-30'
min_days = 18
min_vol = 100000

# Assigning variables to currency to check valid stocks with CAD or USD denominations
currency1 = 'CAD'
currency2 = 'USD'

# loop to check each ticker
for ticker in tickers:
    # Extracts data and history for each stock using yfinance
    t = yf.Ticker(ticker)
    info = t.info
    hist = t.history(start=vol_start_date, end=vol_end_date)
    
    if hist.empty: 
        # Checks if historical data is empty (delisted or unavailable ticker)
        print("Skipped Delisted: ", ticker)

    # Checks if the financial currency is not CAD or USD. Skips invalid currencies
    elif info.get("financialCurrency") != currency1 and info.get("financialCurrency") != currency2:
        print("Skipped non Canadian/American: ", ticker)

    # Checks for duplicate tickers
    elif ticker in valid_tickers:
        print("Skipped Duplicate: ", ticker)

    # Creates monthly data frames for historical data using pandas 
    else:
        monthly_dataframes = [data for _, data in hist.groupby(pd.Grouper(freq='MS'))] # Cited AI usage
        valid_months = [] # initialize a list to store valid months

    # Checks if average monthly volume is over 100,000
        volume = 0
        months_counted = 0
    # loops through each month's data    
        for month in monthly_dataframes:
            
            # Take out months with less than 18 trading days
            if not (len(month) < min_days):
                volume += month['Volume'].sum()
                months_counted += 1

        # Calculates the average trading volume to see if it meets criteria
        if (volume/months_counted) < min_vol:
            print("Skipped (Monthly volume not >= 100,000): ", ticker)
        else:
            valid_tickers.append(ticker)

# Printing the final list of valid tickers
print(valid_tickers)

404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ASDFAASDF.TO?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ASDFAASDF.TO&crumb=fSBNa0Ze7FR
$ASDFAASDF.TO: possibly delisted; no timezone found


Skipped Delisted:  asdfaasdf.to


404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/INVALIDTIC?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=INVALIDTIC&crumb=fSBNa0Ze7FR
$INVALIDTIC: possibly delisted; no timezone found


Skipped Delisted:  INVALIDTIC


$HDFC.NS: possibly delisted; no timezone found


Skipped Delisted:  HDFC.NS


$CELG: possibly delisted; no timezone found


Skipped Delisted:  CELG
Skipped Duplicate:  COST
Skipped non Canadian/American:  GC=F
Skipped Duplicate:  T.TO


$AW.TO: possibly delisted; no price data found  (1d 2023-10-01 -> 2024-09-30) (Yahoo error = "Data doesn't exist for startDate = 1696132800, endDate = 1727668800")


Skipped Delisted:  AW.TO
['AAPL', 'ABBV', 'LOW', 'AUST', 'HOOD', 'AMZN', 'AXP', 'BAC', 'BK', 'SQ', 'VZ', 'CMCSA', 'SHOP', 'COST', 'CSCO', 'CVS', 'GM', 'GOOG', 'JPM', 'IBM', 'ORCL', 'OXY', 'DUOL', 'PEP', 'T.TO', 'SLB', 'SO', 'SPG', 'SHOP.TO', 'RY.TO', 'TD.TO']


# PART 2: Ranking the stocks by running multiple checks

## Combined Index and Individual Stock Weekly Returns Data Preparation

To meet the market, we have to establish a combined benchmark that will act as the index we are trying to meet. Since our returns would be compared to the simple arithmetic average of both the S&P 500 and the TSX 60 indexes combined, we decided to look at weekly returns for both indexes over a certain period of time and then take the average of the two for each week. We then stored the weekly returns of the combined benchmark into a data frame to be used in our analysis after. 

Next, we needed to prepare stock data to compare to the combined benchmark we created. So we decided to create a similar data frame that has the weekly returns for each stock from the same time period. This way, when doing analysis after, we can compare each stock or a combination of stocks against the index based on their weekly returns. 

To simplify the analysis, we chose to represent weekly returns as a percentage instead of value-based returns as percentage-based returns are independent of currency denomination, avoiding the need for currency conversion.

We chose to use data from the past year as it is the most relevant data to current market trends and economic conditions. 1 year of data also includes different earnings seasons, holiday effects, annual economic events and other parts of the market cycle which is sufficient for our analysis. If we used older periods we could include trends that are not relevant today such as a bull market or large amounts of growth in a specific industry. 

We chose to use weekly data as it avoids daily market fluctuations that may be overly volatile or 1-day news events that could heavily affect the performance of a stock. Since we are testing data for a year, weekly data also gives us 52 data points rather than 12 if we were to use monthly data. This gives us better statistical reliability and makes our data less sensitive to outliers. Since we'll be investing for 1 week, it's important to use data within the year, however, if we were looking at a long-term portfolio, we could analyze monthly data instead to get a larger spread of data. 

In [8]:
# list of valid_tickers 
ticker_symbols=valid_tickers

# Assigning tickers for benchmark indicies
benchmark_sp500 = '^GSPC'   # S&P 500 index
benchmark_tsx60 = 'XIU.TO'  # S&P/TSX 60 iShares index ETF

# Time period for historical data 
start_date = '2023-09-01'  
end_date = '2024-09-01'  

# Extracting historical data for benchmarks 
sp500_data = yf.Ticker(benchmark_sp500)
tsx60_data = yf.Ticker(benchmark_tsx60)

# Getting weekly historical data for both indicies 
sp500_hist = sp500_data.history(start=start_date, end=end_date, interval= "1wk")
tsx60_hist = tsx60_data.history(start=start_date, end=end_date, interval= "1wk")

# Calculating the weekly returns for both indicies 
sp500_hist['Weekly Return'] = sp500_hist['Close'].pct_change()
tsx60_hist['Weekly Return'] = tsx60_hist['Close'].pct_change()


# Creates a data frame with the weekly returns of both indexes 
combined_returns = pd.DataFrame({
    'S&P 500 Weekly Return': sp500_hist['Weekly Return'],
    'S&P/TSX 60 Weekly Return': tsx60_hist['Weekly Return']
})

# Creates new row with the calculated average of the weekly returns from both indices
combined_returns['Average Weekly Return'] = combined_returns.mean(axis=1)
combined_returns.index = combined_returns.index.strftime('%Y-%m-%d')


# Initializes a data frame to store weekly returns for all tickers
weekly_returns_df = pd.DataFrame()

# Loops through each ticker to download data and calculate weekly returns
for ticker in ticker_symbols:
    # Download weekly data for the current stock
    stock_data = yf.download(ticker, start=start_date, end=end_date, interval='1wk') # Cited Ai Usage
    
    # Calculate the weekly return based on the close price
    stock_data['Weekly Return'] = stock_data['Close'].pct_change()
    
    # Add the weekly returns as a column in the DataFrame
    weekly_returns_df[ticker] = stock_data['Weekly Return']

weekly_returns_df.index = weekly_returns_df.index.strftime('%Y-%m-%d')

# Aligns the weekly returns and combined benchmark data frames to have the same index
weekly_returns_df, combined_returns_aligned = weekly_returns_df.align(combined_returns['Average Weekly Return'], join='inner', axis=0) # Cited Ai Usage 

# Adds the combined benchmark average returns as a new column in the weekly returns DataFrame
weekly_returns_df['Combined Benchmark'] = combined_returns_aligned

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

# Parameter 1: Tracking Error 

The first parameter we will use to analyze each stock is the tracking error compared to the combined index. Tracking error is a statistical measure used to evaluate how much a stock's returns deviate from the returns of a certain benchmark. It is calculated as the standard deviation of the difference between the stock's returns and the benchmark's returns over a specific period of time. A lower tracking error means the stock's returns closley follow the benchmark. 

Standard deviation is a measure of how spread out a set of values is, showing the average distance of each data point from the mean. In the context of tracking error, it quantifies the typical deviation of a stock's returns from the benchmark's returns.

Since we created a data frame that has the weekly returns of each stock and the combined benchmark over the same time period, we will use that to calculate tracking error. We will then put all the tracking error values into a data frame with the index being the ticker and the second column being the tracking error. The value we are aiming for is as close to 0 as possible since we want to meet the market. The tracking error can't be negative since we are calculating the standard deviation of the difference in returns and standard deviation can not be negative. 

The tracking error is calculated as $\sigma(S - B)$, where $S$ is sequence of stock returns and $B$ is a sequence of benchmark returns over the same time periods.

In [10]:
# Calculate tracking error for each stock using the combined benchmark column

# Initialize a dictionary to store tracking errors of each stock
tracking_errors = {}
for ticker in ticker_symbols:
    # Ensures there are no NaN values in the comparison
    valid_data = weekly_returns_df[[ticker, 'Combined Benchmark']].dropna()
    
    # Calculates the tracking error as the standard deviation of the difference between returns
    tracking_error = np.std(valid_data[ticker] - valid_data['Combined Benchmark'])

    # Stores the calculated tracking error in the dictionary with the Ticker as the key
    tracking_errors[ticker] = tracking_error

# Creates a DataFrame from the tracking errors dictionary
tracking_errors_df = pd.DataFrame.from_dict(tracking_errors, orient='index', columns=['Tracking Error'])

# Sets the index name as Stock
tracking_errors_df.index.name = 'Stock'

# Parameter 2: Beta Value 

The second parameter we will use to analyze each stock is the beta value compared to the combined index. Beta is a statistical measure that evaluates the sensitivity of a stock's returns in relation to the returns of a benchmark. It measures how much a stock's returns are expected to change in response to a change in the benchmark's returns. A beta value of 1 indicates that the stock's returns move in perfect correlation with the benchmark, meaning it follows the benchmark closely.

Since we have a data frame with the weekly returns of each stock and the combined benchmark over the same time period, we can use it to calculate the beta value for each stock. The beta value is calculated as the covariance of the stock's returns and the benchmark's returns divided by the variance of the benchmark's returns:

$$
\beta = \frac{\text{Cov}(S, B)}{\text{Var}(B)}
$$

Where:
- \( S \): Sequence of stock returns.
- \( B \): Sequence of benchmark returns over the same time periods.


Covariance measures the degree to which two variables, such as stock returns and benchmark returns, move together. It is calculated as the average of the product of the deviations of each return from their respective means. A positive covariance means that the two variables move in the same direction, while a negative covariance indicates they move in opposite directions.

Variance, on the other hand, measures the spread of a single variable, such as benchmark returns, around its mean. It quantifies how much the benchmark's returns fluctuate over time and is calculated as the average of the squared deviations from the mean.
me periods.

We will calculate the beta values for all stocks and store them in a data frame, where the index is the stock ticker and the second column is t The value we are looking for is 1, as it indicates that the stock's performance is perfectly in sync with the benchmark. A beta value greater than 1 means the stock is more volatile than the benchmark, while a beta less than 1 indicates it is less volatile. Negative beta values are possible but uncommon, representing an inverse relationship with the benchmark. However, for this analysis, we aim for a beta value close to 1 to ensure the stock tracks the benchmark's performance. 

In [12]:
# Initialize Dictionary to store beta values for each stock
beta_values = {}

# loops through each stock in the weekly returns data frame except Combined_benchmark
for stock in weekly_returns_df.columns[:-1]:
    
    # Drops NaN values for both stock and benchmark
    valid_data = weekly_returns_df[[stock, 'Combined Benchmark']].dropna()
    
    # Calculates covariance between the stock's and benchmark's weekly returns
    covariance = valid_data[stock].cov(valid_data['Combined Benchmark'])
    
    # Calculates variance of the benchmark's weekly returns 
    benchmark_variance = valid_data['Combined Benchmark'].var()
    
    # Calculates the beta value 
    beta = covariance / benchmark_variance

    # Stores the Beta value in the dictionary with the ticker as the key
    beta_values[stock] = beta

# Creates a DataFrame to store beta values
beta_df = pd.DataFrame.from_dict(beta_values, orient='index', columns=['beta'])
beta_df.index.name = 'Stock'

## Parameter 3: Correlation Between a Given Stock and Index
### What is correlation?
.corr() is a Pandas function that calculates Pearson's Correlation Coefficient. PCC finds a pairwise correlation between 2 variables (given as 2 columns in a dataframe). PCC measures the strength and direction of the linear relationship between these variables. It can output a number between -1 to 1, where the sign indicates direction (-1 indicates a negative correlation, +1 indicates positive correlation) and the value indicates strength (if |PCC| = 1 then the correlation between the 2 variables is a perfect linear correlation. If PCC = 0 then there is no correlatio.) The close PCC is to 1, the more exact of a linear relationship.

### Why is it important?
When we find the correlation between a stock and a benchmark (in this case, the index), we find whether the stock moves in the same direction as the index or not. This is important because we want stocks that move similarly to the index to be valued more. Therefore, later on, when we rank the stocks by giving each stock a rating, we can factor correlation into that rating by prioritizing stocks with better correlation.

In [14]:
# finds the correlation of a stock to the given indexes
def high_corr_stocks(stock_lst,indexdf):
    # initializing lists and the DataFrame to be outputted
    high_corr_stock=[]
    high_corr_num=[]
    corr_df=pd.DataFrame(index=stock_lst, columns=['correlation'], dtype='float64')

    i=0
    while i<len(stock_lst):
        # creates a DF with a column for the index average found above
        df=pd.DataFrame()
        df['index avg']=indexdf['Average Weekly Return']
        # adds a column to the DF for the stock
        tick=yf.Ticker(stock_lst[i])
        df[stock_lst[i]]=tick.history(start=start_date,end=end_date,interval='1wk')['Close'].pct_change()
        df=df.dropna()

        # finds their correlation and adds it to the dataframe
        corr=df.corr()
        corr_df.loc[stock_lst[i],'correlation']=corr.loc[stock_lst[i],'index avg']
        i+=1

    return corr_df

# using the function on the given list of stocks
corr_df=high_corr_stocks(list(beta_df.index),combined_returns.dropna())

## Ranking Stocks
Based on the 3 main tests above (tracking error, beta, and correlation), we assign each stock a score. This is done by first normalizing the outputs for each test via Max-Min Normalization (MMN). MMN is a form of normalization that brings a data set to a range of 0-1, where the lowest value in the set is 0 and the highest value is 1. We do this for each test so we can get their outputs onto the same range (since each test can return different value ranges). 

Once this is done, we assign each stock a score. This is done by summing together the normalized outputs for each test in an equally weighted fashion. This score will always be between 0-1: scores closer to 1 means the stock is a better match to the benchmark (the combined indices). Using these scores, we can now rank these stocks in order of best to worst.

In [16]:
# tracking is the DataFrame created by the function that calculates tracking error
# beta is the DataFrame created by the function that calculates tracking error
# corr is the DataFrame created by the function that calculates correlation
# los is the list of stocks
    # los should be the same as the index column on tracking and beta and correlation dataframes
def normalize_scores(tracking,beta,corr,los):
    # creating a dataframe that uses los as an index
    rankeddf=pd.DataFrame(index=los)
    df_track=tracking.copy(deep=True)
    df_beta=beta.copy(deep=True)
    df_corr=corr.copy(deep=True)

    # normalizing tracking and beta scores via max normalization
    df_track['Tracking Error']=(tracking['Tracking Error']-(tracking['Tracking Error'].min()))/((tracking['Tracking Error'].max())-(tracking['Tracking Error'].min()))
    df_beta['beta']=abs(1-df_beta['beta'])
    df_beta['beta']=(df_beta['beta']-df_beta['beta'].min())/(df_beta['beta'].max()-df_beta['beta'].min())
    df_corr['correlation']=abs(1-df_corr['correlation'])
    df_corr['correlation']=(df_corr['correlation']-df_corr['correlation'].min())/(df_corr['correlation'].max()-df_corr['correlation'].min())
    
    # initializing a list and iterator for the loop
    score_lst=[]
    i=0
    # this loop will create a score for each stock based off of the normalized tracking, beta, and correlation scores
    # these scores are equally weighted and each score will be between 0-1
    while i<len(los):
        item_score=((df_track.loc[los[i],'Tracking Error'])*(1/3))+((df_beta.loc[los[i],'beta'])*(1/3))+((df_corr.loc[los[i],'correlation'])*(1/3))
        score_lst.append(item_score)
        i+=1

    # adding the scores to the dataframe and ranking them so the best score is on top
    rankeddf['score']=score_lst
    rankeddf['score']=1-rankeddf['score']
    rankeddf=rankeddf.sort_values(by='score',ascending=False)
    return rankeddf

# ranking the given list of tickers
ranked=normalize_scores(tracking_errors_df,beta_df,corr_df,list(tracking_errors_df.index))
ranked

,score
RY.TO,0.988152
JPM,0.951305
BK,0.943964
TD.TO,0.878317
AXP,0.857022
AMZN,0.850516
BAC,0.840386
SPG,0.838550
LOW,0.818967
CMCSA,0.794069


# PART 3: Finding the Optimal Portfolio

## HELPER FUNCTION: Weighting a given portfolio
### How do we weigh each stock in the portfolio?
Given a dataframe of ranked stocks, we can begin to assign weights. The first thing we apply are the base rules from the project outline. Each stock must weigh at least 100/2n% of the portfolio (where n is the number of stocks in the portfolio). Once the base weights have been distributed, we can now distribute the rest of the portfolio weight based on stock scores.

We distribute portfolio weight by using the stock scores as a way to weigh each stock; the higher the stock score, the greater its portion of the portfolio. Our equation to find the weight of each stock is as follows:
$$\frac{x_i}{sum(X)}$$
where $x_i$ is the score of each stock and $X$ is the list of stock scores.

By weighing our stocks based on how well they scored on our parameters, we optimize our portfolio so that the best stocks are weighed much more than the under-performing stocks.

Finally, we ensure that no stock is weighted more than 15%. This is done by taking whatever excess weight has been distributed to the stocks weighed more than 15% and passing it to the first stock that has less than 15% weight in the portfolio (and so on until all the weight has been redistributed). Since the stocks are given in ranked order, the weight will always be redistributed to the best stock weighted under 15%. This way, we maintain the skew of good stocks being a higher weight in the portfolio than bad stocks.

In [19]:
# this portfolio determines the weighting of the portfolio based on the ranked scores
# each stock will be weighted as the percentage of its score out of the sum of all the scores
    # ie) stock_weight = stock_score/sum(list_of_stocks)
def pf_weighting(rankeddf,score_lst):
    # initializing a dataframe for stock weights
    pf_weight=pd.DataFrame(index=score_lst)

    sum_scores=rankeddf['score'].sum()
    # base weight is the minimum weight all stocks must have
    base_weight=1/(2*len(score_lst))

    # initializing list for weights and iterator
    weight_lst=[]
    close_price=[]
    currency_lst=[]
    i=0
    # function will create a list of weights for the given list of stocks
    while i<len(score_lst):
        # adds the new weight (calculated by weighting the ranking) to the base weight
        item_weight=(((rankeddf.loc[score_lst[i],'score'])/sum_scores)*0.5)+base_weight  #because after the base weights have been distributed only 50% is left
        weight_lst.append(item_weight)

        # pulls the close price for each stock
        tick=yf.Ticker(score_lst[i])
        close=tick.history(start="2024-11-22", end="2024-11-23")['Close'].iloc[0]
        close_price.append(close)

        # pulls the currency for each stock
        ticker=yf.Ticker(score_lst[i])
        currency=ticker.fast_info['currency']
        currency_lst.append(currency)
        i+=1

    # putting the list of weights into the dataframe
    pf_weight['weight']=weight_lst
    pf_weight['close']=close_price
    pf_weight['currency']=currency_lst

    # redistributes the stock weight if any stock is over 15% of the portfolio
    if pf_weight.iloc[0,0]>0.15:
        temp=0
        i=0
        while i<len(score_lst):
            if pf_weight.loc[score_lst[i],'weight']>=0.15:
                temp=temp+(pf_weight.loc[score_lst[i],'weight']-0.15)
                pf_weight.loc[score_lst[i],'weight']=0.15
            elif temp+pf_weight.loc[score_lst[i],'weight']<0.15:
                pf_weight.loc[score_lst[i],'weight']=temp+pf_weight.loc[score_lst[i],'weight']
                temp=0
            else:
                temp=temp-(0.15-pf_weight.loc[score_lst[i],'weight'])
                pf_weight.loc[score_lst[i],'weight']=0.15
            i+=1
            
    return pf_weight

## HELPER FUNCTION: Calculating Fees, Number of Stocks to Buy & Pulling Info from yFinance

Since we might have stocks in both CAD and USD, we need to appropriately calculate the number of stocks to buy. To do so, we convert the close price from USD to CAD and calculate from there. We also calculate which fee to pay and how that affects the 1,000,000 budget we have. Our fee options were to pay a flat fee of 3.95 or a variable fee of 0.001 per share. We calculated the number of shares we could buy given both fees and took the larger share count. Finally, we pull stock information, such as currency and close price, directly from yFinanc

.
.


In [21]:
# this function will calculate the fees paid on each stock, the number of stocks bought given its weight, 
# each stock's close price & currency, and calculates how much each stock weighs of 1 million

# weight_df is the datadrame with all the decimal weights of the given stocks (if stock A takes up 10% of the portfolio the weight is given as 0.1)
def stock_info(weight_df,stock_lst):
    # finding currency conversion
    usd_to_cad_ticker=yf.Ticker("CADUSD=X") # NEW CODE
    usd_to_cad_rate=usd_to_cad_ticker.history(start="2024-11-22",end="2024-11-23")['Close'].iloc[0]

    # initializing lists to store info
    stock_final_weights=pd.DataFrame(index=stock_lst)
    stock_num=[]
    fees=[]
    acc_stock_weight=[]
    i=0
    while i<len(stock_lst):
        # finds the amount of cash we can spend on a given stock in CAD
        cost=1000000*weight_df.loc[stock_lst[i],'weight']
        # finds the close of the current stock in the loop
        close=weight_df.loc[stock_lst[i],'close']

        # converts stock close from USD to CAD
        if weight_df.loc[stock_lst[i],'currency']=='USD':
            close=weight_df.loc[stock_lst[i],'close']/usd_to_cad_rate

        # fixed_snum will return the number of stocks we could buy given that the fee is the fixed $3.95
        fixed_snum=(cost-3.95)/close
            
        # dyn_snum will return the number of stocks we could buy given that the fee is the dynamic $0.001 per stock
        dyn_snum=cost/(0.001+close)
    
        # if/else conditions will ensure the max amount of stocks is bought given the different fee options (allows us to keep the stock weight as close to original as possible
        if fixed_snum>=dyn_snum:
            stock_num.append(fixed_snum)
            fees.append(3.95)
        else:
            stock_num.append(dyn_snum)
            fees.append(stock_num[i]*0.001)
    
        # finds the new weight of the stock given that a portion of the original weight is dedicated to paying the trading fees
        new_weight=((stock_num[i]*weight_df.loc[stock_lst[i],'close'])/cost)*weight_df.loc[stock_lst[i],'weight']
        acc_stock_weight.append(new_weight)
        i+=1

    # appending columns to the dataframe
    stock_final_weights['Stock weight']=weight_df['weight']
    stock_final_weights['# stocks bought']=stock_num
    stock_final_weights['Close']=weight_df['close']
    stock_final_weights['fees']=fees
    stock_final_weights['Stock weight of 1 mil']=acc_stock_weight
    stock_final_weights['currency']=weight_df['currency']

    return stock_final_weights

## Optimizing the number of stocks in our portfolio

We optimize the number of stocks in our portfolio by using tracking error. The process is as follows:
* we cycle through portfolios made up of 12-24 of our top scored stocks
    * for each portfolio, we run a tracking error test against the benchmark
        * the weight of each stock in each portfolio is calculated by the pf_weighting function from above
    * the portfolio with the lowest tracking error is picked as our optimal number of stocks
* from there, we run stock_info to begin pulling the necessary information for our final dataframe.

In [23]:
# Pull the USD to CAD exchange rate using yfinance
usd_to_cad_ticker=yf.Ticker("CADUSD=X") # NEW CODE
usd_to_cad_rate=usd_to_cad_ticker.history(start="2024-11-22",end="2024-11-23")['Close'].iloc[0]
# finds the optimal number of stocks to put into the portfolio by looking at which number of stocks has the lowest tracking error
def optimal_stock_num(score_df,weekly_returns_df):
    # initializing variables
    best_tracking_error=float('inf')
    best_stock_num=0
    # iterator starts at 12 because that's the minimum number of stocks we can pick
    i=12
    while i<25:
        # creates a dataframe that only has i amount of stocks and finds their weights
        temp_score_df0=score_df.head(i)
        temp_score_df=pf_weighting(temp_score_df0,list(temp_score_df0.index))

        # Calculate portfolio weekly returns by summing weighted returns of selected stocks
        portfolio_returns=weekly_returns_df[temp_score_df.index].mul(temp_score_df['weight'].values, axis=1).sum(axis=1)
        # Calculate tracking error (standard deviation of differences)
        tracking_error=np.std(portfolio_returns-weekly_returns_df['Combined Benchmark'])

        # finds the best tracking error
        if abs(tracking_error)<abs(best_tracking_error):
            best_tracking_error=tracking_error
            best_stock_num=i
            
        i+=1

    # creating the optimal portfolio
    optimal_score_df=score_df.head(best_stock_num)
    best_port0=pf_weighting(optimal_score_df,list(optimal_score_df.index))
    best_port=stock_info(best_port0,list(best_port0.index))
    final_stock_lst=list(best_port.index)
    
    # Adjust prices and values for USD stocks using the exchange rate
    adjusted_values=[]
    i=0
    while i<len(final_stock_lst):
        value=best_port.loc[final_stock_lst[i],'Close']*best_port.loc[final_stock_lst[i],'# stocks bought']
        if best_port.loc[final_stock_lst[i],'currency']=='USD':
            value=(best_port.loc[final_stock_lst[i],'Close']/usd_to_cad_rate)*best_port.loc[final_stock_lst[i],'# stocks bought']
        adjusted_values.append(value)
        i+=1

    # Rebuild the final portfolio with adjusted values and values
    final_port = pd.DataFrame({
    'Ticker': final_stock_lst,
    'Price': best_port['Close'],
    'Currency': best_port['currency'],
    'Shares': best_port['# stocks bought'],
    'Value': adjusted_values,
    'Weight': best_port['Stock weight']
    })

    fees_total=best_port['fees'].sum()
    value_total=(final_port['Value'].sum())+fees_total
    print("The total fees are $",np.round(fees_total,2))
    print("The total value of the portfolio is $",np.round(value_total,2))
    weight_total=(final_port['Weight'].sum())
    print("The sum of all stock weights in the portfolio is",weight_total)
    
    # reindexing the portfolio
    final_port.index=np.arange(1,len(final_port)+1)
    return final_port

In [24]:
Portfolio_Final=optimal_stock_num(ranked,weekly_returns_df)
Portfolio_Final

The total fees are $ 9.43
The total value of the portfolio is $ 1000000.0
The sum of all stock weights in the portfolio is 1.0000000000000002


,Ticker,Price,Currency,Shares,Value,Weight
1,RY.TO,174.710007,CAD,321.242182,56124.223705,0.056125
2,JPM,248.550003,USD,158.181436,54963.762060,0.054964
3,BK,80.139999,USD,488.524853,54732.232421,0.054733
4,TD.TO,78.510002,CAD,670.797477,52664.311384,0.052665
5,AXP,301.299988,USD,123.437743,51994.122387,0.051994
6,AMZN,197.119995,USD,187.931963,51789.115446,0.051789
7,BAC,47.000000,USD,783.329832,51469.469174,0.051470
8,SPG,181.139999,USD,203.022769,51412.209996,0.051412
9,LOW,264.679993,USD,137.276628,50795.458265,0.050796
10,CMCSA,43.470001,USD,822.933435,50010.537770,0.050011


In [25]:
# creating the final dataframe that outputs to csv
Stocks_Final = Portfolio_Final[['Ticker', 'Shares']].copy()
Stocks_Final.to_csv('Stocks_Group_14.csv', index=False)
Stocks_Group_14 = pd.read_csv('Stocks_Group_14.csv')
Stocks_Group_14.index = Stocks_Group_14.index + 1
Stocks_Group_14.to_csv('Stocks_Group_14.csv', index_label='Index')
display(Stocks_Group_14)

,Ticker,Shares
1,RY.TO,321.242182
2,JPM,158.181436
3,BK,488.524853
4,TD.TO,670.797477
5,AXP,123.437743
6,AMZN,187.931963
7,BAC,783.329832
8,SPG,203.022769
9,LOW,137.276628
10,CMCSA,822.933435
